In [1]:
import json
import requests
from src import utils
import time
import os
import csv
import re

### Making Training set

In [2]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'
# input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'

In [3]:
data = utils.read_jsonlines(input_file)

In [4]:
len(data)

1319

In [5]:
PROMPT1 = "질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? \
풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. \
정답: 72 \\n"


In [6]:
def get_gsm8k_zs(que):
    prompt = f'질문 : {que} \n정답 : '
    return prompt

In [7]:
def get_gsm8k_build_data_with_reason(kor_que, kor_ans, ans_s):
    prompt = f"질문: {kor_que}"
    output = f"풀이: {kor_ans} 정답: {ans_s}"
    return prompt, output

In [18]:
def get_gsm8k_build_data_with_one_shot_cot_finetuning(kor_que, kor_ans, ans_s, PROMPT1):
    PROMPT1 
    prompt = f"질문: {kor_que} 풀이: "
    output = f"{kor_ans} 정답: {ans_s}"
    
    prompt = PROMPT1 + prompt
    return prompt, output

In [19]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [20]:
# from pprint import pprint

In [21]:
def remove_line_slit(context):
    context = re.sub(r"\n", " ", context)
    return context

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [22]:
# cnt = 0
split_cnt = 0
input_lst = []
output_lst = []

for ins in data:
    
    
    kor_que = ins['kor_question']
    kor_ans = ins['kor_answer']
    ans_s = ins['answer_s']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue
    
#     print(f'kor_que')
#     print(f'{kor_que}')
#     print(f'kor_ans')
#     print(f'{kor_ans}')
#     print(f'ans_s')
#     print(f'{ans_s}')
    
    kor_ans = ins['kor_answer']
    in_, out_ = get_gsm8k_build_data_with_one_shot_cot_finetuning(kor_que, kor_ans, ans_s, PROMPT1)
    
    
    input_lst.append(remove_line_slit(in_))
    output_lst.append(remove_line_slit(out_))

#     if cnt == 5:
#         break
#     else:
#         cnt += 1

In [23]:
len(data) - split_cnt

1293

In [24]:
len(input_lst)

1293

In [25]:
len(output_lst)

1293

In [36]:
input_lst[10]

'질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \\n질문: 툴라는 빵집에 가서 다양한 종류의 과자를 샀습니다. 그녀는 3다스의 도넛을 1다스당 68달러에, 2다스의 미니 컵케이크를 1다스당 80달러에, 그리고 6다스의 미니 치즈케이크를 1다스당 55달러에 샀습니다. 총 비용은 얼마인가요? 풀이: '

In [37]:
output_lst[10]

'도넛의 총 비용은 3 x $68 = $204였습니다. 미니 컵케이크의 총 비용은 2 x $80 = $160이었습니다. 미니 치즈케이크의 총 비용은 6 x $55 = $330이었습니다. 따라서 Toula가 과자를 구입한 총 비용은 $204 + $160 + $330 = $694입니다. 정답: 694'

In [38]:
def save_csv_file(file_path, sentence_a_list, sentence_b_list):
    with open(file_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        for sentence_a, sentence_b in zip(sentence_a_list, sentence_b_list):
            writer.writerow([sentence_a, sentence_b])

In [39]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [40]:
output_file = output_path + '/kor-gsm8k-one_shot_cot_finetuning-train.csv'

In [41]:
output_file

'/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-one_shot_cot_finetuning-train.csv'

In [42]:
save_csv_file(output_file, input_lst, output_lst)

In [43]:
input_lst[1]

'질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \\n질문: 조쉬는 집을 뒤집어 판매해보기로 결정합니다. 그는 $80,000에 집을 구매하고 $50,000을 수리합니다. 이로 인해 집의 가치가 150% 증가합니다. 그가 얼마의 이익을 얻었습니까? 풀이: '

In [44]:
output_lst[1]

'집과 수리비용은 80,000+50,000=$130,000이 나왔습니다. 그는 집의 가치를 80,000*1.5=120,000만큼 높였습니다. 그래서 집의 새로운 가치는 120,000+80,000=$200,000입니다. 그래서 그는 200,000-130,000=$70,000의 이익을 냈습니다. 정답: 70000'

### Making Test Set

In [46]:
output_file = output_path + '/kor-gsm8k-one_shot_cot_finetuning-test.jsonl'

In [47]:
print(output_file)

/userhomes/philhoon/kt-ai-challenge/result/kor-gsm8k-one_shot_cot_finetuning-test.jsonl


In [48]:
jsonl_writer = utils.JSONLWriter(output_file)

In [49]:
len(data)

1319

In [51]:
# cnt = 0
split_cnt = 0
input_lst = []
output_lst = []

for ins in data:
    
    
    kor_que = ins['kor_question']
    kor_ans = ins['kor_answer']
    ans_s = ins['answer_s']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue
    
    kor_ans = ins['kor_answer']
    in_, out_ = get_gsm8k_build_data_with_one_shot_cot_finetuning(kor_que, kor_ans, ans_s, PROMPT1)
    print('in_')
    print(f'{in_}')
    print('out_')
    print(f'{out_}')
    
    ins['kor_question_reason'] = in_
    ins['kor_question_reason_answer'] = out_

    jsonl_writer.write_json_line(ins)

in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 한 벌의 로브를 만드는 데 파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요합니다. 총 몇 덩이의 섬유가 필요한가요? 풀이: 
out_
하얀 섬유로 만든 볼트 1개가 필요합니다.
그러므로 전체 원단 양은 2+1=3개의 볼트입니다. 정답: 3
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 조쉬는 집을 뒤집어 판매해보기로 결정합니다. 그는 $80,000에 집을 구매하고 $50,000을 수리합니다. 이로 인해 집의 가치가 150% 증가합니다. 그가 얼마의 이익을 얻었습니까? 풀이: 
out_
집과 수리비용은 80,000+50,000=$130,000이 나왔습니다.
그는 집의 가치를 80,000*1.5=120,000만큼 높였습니다.
그래서 집의 새로운 가치는 120,000+80,000=$200,000입니다.
그래서 그는 200,000-130,000=$70,000의 이익을 냈습니다. 정답: 70000
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 제임스는 매주 3번씩 3번의 짧

in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 브리타니와 그녀의 엄마는 박물관에 갑니다. 입장료는 어른은 12달러, 어린이는 10달러입니다. 브리타니의 엄마는 어린이 티켓 1장과 어른 티켓 1장을 위해 캐셔에게 돈을 줍니다. 만약 그녀가 8달러의 거스름돈을 받았다면, 그녀는 얼마나 많은 돈을 캐셔에게 주었습니까? 풀이: 
out_
티켓의 총 비용은 12+10=22달러입니다.
브리타니 어머니께서 캐셔에게 22+8=30달러를 주셨습니다. 정답: 30
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 찰리는 도로시의 세 배의 페이스북 친구를 가지고 있습니다. 제임스는 도로시의 네 배의 페이스북 친구를 가지고 있습니다. 찰리가 페이스북에서 12명의 친구를 가지고 있다면, 제임스는 몇 명의 페이스북 친구를 가지고 있을까요? 풀이: 
out_
도로시는 페이스북 친구가 12 / 3 = 4명 있습니다.
제임스는 페이스북 친구가 4 * 4 = 16명 있습니다. 정답: 16
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 스티브와 팀은 누가

in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 수잔은 그녀가 하는 온라인 작업마다 10분마다 $5를 벌입니다. 만약 그녀가 오전 8시부터 11시까지 일하고, 그 사이에 30분씩 쉰다면, 온라인 작업으로 얼마나 돈을 벌까요? 풀이: 
out_
그녀가 30분 동안 쉬기 때문에, 그녀가 일하는 시간은 11 - 8 - 0.5 = 2.5 시간입니다.
그래서 그녀는 2.5 * 60 = 150 분을 일합니다.
그녀는 10분당 $5를 벌기 때문에, 온라인 작업으로 그녀는 총 150 * 5/10 = $75를 벌게 됩니다. 정답: 75
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 마이크, 짐, 토니라는 세 친구가 게임을 하기로 결정했습니다. 3 라운드 후에 마이크는 21 점, 짐은 마이크보다 3 점 적고, 토니는 마이크의 2 배입니다. 네 번째 라운드에서, 20 점 이상인 경우 각 플레이어는 추가 점수를 받습니다. 추가 점수가 배분된 후에 세 플레이어가 모두 얻은 총 점수는 얼마입니까? 풀이: 
out_
라운드가 3번 지난 후, 짐은 21점 - 3점 = 18점을 가지고 있습니다.
토니는 마이크의 점수보다 2배 더 많은 점수를 가지고 있으므로, 2 * 21점 = 42점을 가지고 있습니다.
그들은 총 42점 + 18점 + 21점 = 81점을 가지고 있습니다.
20점 이상의 점수를 가진 사람은 두 명뿐이므로, 모든 친구

in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 프린스 타데우스는 강력한 검으로 100마리의 용을 죽였고, 프린스 아서는 창을 사용하여 타데우스 왕자의 3/4만큼의 용을 죽였습니다. 프린스 월터는 낡은 철 도끼를 사용하여 아서 왕자의 두 배만큼의 용을 죽였습니다. 그러나 검을 집에 두고 온 프린스 브루스는 못깍이로 월터 왕자의 1/5만큼의 용을 죽였습니다. 프린스 브루스는 몇 마리의 용을 죽였습니까? 풀이: 
out_
아서 왕자는 타데우스 왕자의 3/4만큼 용을 잡았습니다. 즉, 100*(3/4)=75마리의 용을 잡았습니다.
월터 왕자는 아서 왕자의 두 배인 2*75=150마리의 용을 잡았습니다.
브루스 왕자는 월터 왕자의 1/5만큼 용을 잡았습니다. 즉, 150/5=30마리의 용을 잡았습니다. 정답: 30
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 15 갤런의 가솔린이 5개의 다른 용기에 동일하게 분배되었습니다. Josey는 그녀의 잔디 깎는 기계를 작동시키기 위해 용기의 1/4가 필요했습니다. Josey가 필요로 한 가솔린의 파인트 수는 얼마인가요? 풀이: 
out_
조지는 그녀의 잔디 깎는 기계에 6파인트의 가스가 필요했습니다. 정답: 6
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 

in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 푸드 트럭은 그릴 치즈만 판매합니다. 그들은 빵을 1개에 $3.00에 구매하며, 1개의 빵으로 10개의 샌드위치를 만듭니다. 그들은 10개의 샌드위치 당 $30.00을 치즈와 양념에 사용합니다. 만약 그들이 10개의 샌드위치를 각각 $7.00에 판매한다면, 그들의 순이익은 얼마인가요? 풀이: 
out_
그들은 빵에 $3.00을, 치즈와 양념에 $30.00을 사용하여 총 $33.00을 썼습니다.
그들은 한 개당 $7.00에 10개의 샌드위치를 판매하여 총 $70.00을 벌었습니다.
그들은 $70.00을 벌고 $33.00을 썼으므로, 10개의 샌드위치 당 순 이익은 $37.00입니다. 정답: 37
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 학년이 끝나는 날, 케이트는 선생님께 64색의 크레용이 든 상자 3개를 달라고 부탁했습니다. 모두 작은 조각으로 닳아 있었습니다. 선생님은 동의했습니다. 케이트는 크레용을 집으로 가져가 크레용 머핀을 만들기로 결심했습니다. 그녀는 머핀 틀에 작은 크레용 조각 8개를 녹여 오븐에 몇 분 동안 구웠습니다. 그녀가 머핀 크레용을 1.50달러에 판매한다면, 케이트는 얼마나 많은 돈을 벌 수 있을까요? 풀이: 
out_
그녀는 64색연필이 들어있는 상자를 3개 구입하여 3*64 = 192개의 색연필을 가지고 있습니다.
한 개의

in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 제임스는 블루베리를 따기로 결정합니다. 따기 위해서는 $20이 들고, 그 후에는 파운드당 $1.5가 추가됩니다. 그는 30 파운드의 블루베리를 따었습니다. 상점에서 파운드당 $2.5에 사는 것과 비교하여 그는 얼마나 절약했을까요? 풀이: 
out_
그는 딸기를 따기 위해 30*1.5=$45를 지불했습니다.
따라서 입장료를 제외하면 45+20=$65입니다.
만약 식료품점에서 딸기를 산다면 30*2.5=$75가 들 것입니다.
그래서 그는 75-65=$10을 절약했습니다. 정답: 10
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 어떤 회사가 500대의 컴퓨터를 구매하고 각 컴퓨터에 700달러를 지불할 예정이었습니다. 그러나 주문하러 가서 각 컴퓨터의 가격이 예상한 것보다 10% 더 비싸다는 것을 깨달았습니다. 새로운 가격으로 컴퓨터를 구매하기 위해 지불한 총 금액을 계산하세요. 풀이: 
out_
그들이 주문하러 갔을 때, 회사는 각 컴퓨터의 가격이 700달러에 10/100*700 = 70달러씩 올랐음을 깨달았습니다.
새로운 가격으로 총 구매 가격은 700달러 + 70달러 = 770달러입니다.
그들은 500대의 컴퓨터를 구매하고자 했으므로, 총 770달러 * 500대 = 385000달러를 지불했습니다. 정답: 385000
in_
질문: 나탈리아는 4월에 친구

in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: KozyInn 모텔에는 90개의 방이 있습니다. 하우스키핑은 각 방을 청소하는 데 20분이 걸립니다. 방의 절반을 청소하는 데 몇 시간이 걸릴까요? 풀이: 
out_
코지인 모텔의 45개의 방을 청소하는 데는 15시간이 걸릴 것입니다. 90 * 20 = 1800분, 1800/2 = 900분, 900분 = 15시간입니다. 정답: 15
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 지역 마을은 확장하고 다음 3년 동안 여러 새 집을 지으려고 합니다. 첫 해에는 12개의 집을 지을 것입니다. 다음 해에는 이 수의 3배만큼의 집을 지을 것입니다. 세 번째 해에는 지어진 집의 수를 세고 그 수를 두 배로 늘릴 것입니다. 이 마을은 다음 3년 동안 몇 채의 집을 지을까요? 풀이: 
out_
년도별로 말하자면, 두 번째 해에는 첫 해의 3배인 12 * 3 = 36개의 주택을 건설합니다.
지금까지 두 해 동안 36+12 = 48개의 주택을 건설했습니다.
세 번째 해에는 첫 두 해에 건설된 주택 수를 두 배로 늘리므로, 48 * 2 = 96개의 주택을 건설합니다.
모두 합치면, 지역 마을은 48 + 96 = 144개의 주택을 건설했습니다. 정답: 144
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는

in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 에어컨은 저속으로 시간당 2도, 고속으로 시간당 3도의 속도로 방을 냉각합니다. 에어컨을 사용하지 않으면 방은 시간당 2도씩 데워집니다. 에어컨은 3시간 동안 저속으로 작동한 후 4시간 동안 고속으로 작동했습니다. 그 후 2시간 동안 꺼졌습니다. 시작 온도보다 최종 온도가 몇 도 낮아졌습니까? 풀이: 
out_
에어컨은 낮은 세기로 3시간 동안 작동하여 온도가 2 * 3 = 6도 떨어졌습니다.
그 후 높은 세기로 4시간 동안 작동하여 온도가 또 3 * 4 = 12도 떨어졌습니다.
그런 다음 2시간 동안 꺼졌으므로 온도가 2 * 2 = 4도 오르게 되었습니다.
따라서 최종 온도는 시작 온도보다 6 + 12 - 4 = 14도 낮았습니다. 정답: 14
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 할리 혜성은 75년마다 태양 주위를 공전합니다. 빌의 아버지는 혜성을 본 적이 있으며 그 당시 30살이었습니다. 빌은 아버지가 혜성을 본 나이의 3배가 되었을 때 혜성을 두 번째로 본 적이 있습니다. 빌이 처음으로 혜성을 본 나이는 얼마였나요? 풀이: 
out_
빌은 90세에 두 번째로 혜성을 보았습니다. 혜성 할리는 75년마다 볼 수 있으므로, 빌은 15세 때 처음으로 혜성을 보았습니다. 정답: 15
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고,

in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 아놀드, 매들린, 캠던, 사라는 부활절 달걀 56개를 그렸습니다. 아놀드와 매들린은 같은 수의 달걀을 그렸습니다. 캠던과 사라는 총 30개의 달걀을 그렸지만, 캠던은 사라보다 12개를 더 그렸습니다. 캠던은 아놀드보다 몇 개의 달걀을 더 그렸나요? 풀이: 
out_
아놀드와 매들린은 부활절 달걀 56 - 30 = 26개를 그렸습니다.
아놀드와 매들린은 각각 26/2 = 13개의 부활절 달걀을 그렸습니다.
사라는 (30 - 12)/2 = 18/2 = 9개의 부활절 달걀을 그렸습니다.
캠던은 9 + 12 = 21개의 부활절 달걀을 그렸습니다.
캠던은 아놀드보다 21 - 13 = 8개의 달걀을 더 그렸습니다. 정답: 8
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 닉은 크레용 200개를 가지고 있습니다. 그는 8개씩 그룹으로 나누어 상자에 넣고 싶습니다. 각 상자는 8온스의 무게가 있습니다. 각 크레용은 1온스의 무게가 있습니다. 그가 모든 크레용을 상자에 넣으면, 1파운드는 16온스이므로, 상자와 크레용의 총 무게는 얼마입니까? 풀이: 
out_
먼저 Nik가 필요로 하는 상자의 수를 찾으세요. 200개의 크레용 ÷ 상자 당 8개의 크레용 = 25개의 상자.
25개의 상자 × 상자 당 8온스 = 200온스.
각 크레용은 1온스이므로, 200 + 200 =

in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 팀은 수영장 파티를 위해 레모네이드를 만들고 싶어했습니다. 1 갤런의 레모네이드를 만들기 위해서는 그의 레시피에 신선한 레몬 주스 1 컵이 필요합니다. 그는 6 개의 레몬으로 1 컵의 주스를 얻을 수 있다는 것을 발견했습니다. 그는 파티를 위해 4 갤런의 레모네이드를 만들어야 할 것으로 생각했습니다. 그의 가장 친한 친구인 앨런은 다른 갤런의 2 배나 새콤한 레모네이드를 추가로 만들 수 있는지 물었습니다. 팀은 몇 개의 레몬이 필요할까요? 풀이: 
out_
팀은 다른 것들의 2배만큼 새콤한 1 갤런을 만들 것입니다. 각각의 갤런에는 1컵의 주스가 필요합니다. 그래서 그는 새콤한 배치에 2*1 = 2컵이 필요합니다. 
게다가, 그는 레몬에이드를 4개의 일반 갤런을 만들 것입니다. 각각은 1컵의 주스가 필요하므로 총 4컵이 필요합니다. 
모두 합치면, 그는 2 + 4 = 6컵의 주스가 필요합니다. 
각 컵당 6개의 레몬이 필요하고, 그는 6컵의 주스가 필요하므로 6 * 6 = 36개의 레몬이 필요합니다. 정답: 36
in_
질문: 나탈리아는 4월에 친구 48명에게 클립을 팔았고, 그리고 5월에는 그 절반만큼을 팔았습니다. 나탈리아는 4월과 5월에 모두 몇 개의 클립을 팔았을까요? 풀이: 나탈리아는 5월에 48/2 = 24개의 클립을 판매했습니다. 나탈리아는 4월과 5월에 모두 48+24 = 72개의 클립을 판매했습니다. 정답: 72 \n질문: 케넬 하우스는 3마리의 독일 목격개와 2마리의 불독을 기르고 있습니다. 독일 목격개는 하루에 5킬로그램의 개사료를 먹고, 불독은 하루에 3킬로그램의 개사료를 먹습니다. 일주일 동안 얼마나 많은 양의

In [ ]:
len(data) - split_cnt

In [ ]:
split_cnt